<a href="https://colab.research.google.com/github/omkarpat/EmpatheticDialoguesEmotionDetection/blob/master/Complement_Emotion_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Colab settings/mount
from google.colab import drive
drive.mount('/content/gdrive')
%cd gdrive/My\ Drive/CSE\ 245\ Project

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive
/content/gdrive/My Drive/CSE 245 Project


In [4]:
!ls Data

'Data Exploration.ipynb'
 data_fixed_train.json
 data_sample_100.json
 data_sample_10.json
 data_sample_10_processed.json
 data_sample_fixed_processed_final.json
 data_sample_fixed_processed_model1_final.csv
 data_sample_fixed_processed_model2_final.csv
 data_sample_fixed_processed_model3_final.csv
 data_sample_fixed_processed_model4_final.csv
 fixed
 fixed_test.csv
 fixed_test.json
 fixed_train_516.csv
 fixed_valid.csv
 fixed_valid.json
'informative words.ipynb'
 model1_predictions_test.csv
 model1_predictions_train.csv
 model1_predictions_valid.csv
 model2_predictions_test.csv
 model2_predictions_train.csv
 model2_predictions_valid.csv
 model3_predictions_test.csv
 model3_predictions_train.csv
 model3_predictions_valid.csv
 model4_predictions_test.csv
 model4_predictions_train.csv
 model4_predictions_valid.csv
'Model Metrics.gsheet'
 Raw
 test_fixed_processed_final.json
 test_fixed_processed_model1_final.csv
 test_fixed_processed_model2_final.csv
 test_fixed_processed_model3_final.cs

In [2]:
!pip install transformers

     |████████████████████████████████| 675kB 6.1MB/s 
     |████████████████████████████████| 1.1MB 15.5MB/s 
     |████████████████████████████████| 3.8MB 30.9MB/s 
     |████████████████████████████████| 890kB 59.4MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893260 sha256=bd0c863635f786938421dce3bd260d35edecb5830911125b7dcea4a6201c1259
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [0]:
dataset_name = ""
speaker_model_name = "model3"
listener_model_name = "model4"
model_type = "emotion" #pos_neg or emotion

In [3]:
import tensorflow as tf

# Get the GPU device name.
device_name = tf.test.gpu_device_name()

# The device name should look like the following:
if device_name == '/device:GPU:0':
    print('Found GPU at: {}'.format(device_name))
else:
    raise SystemError('GPU device not found')

import torch

# If there's a GPU available...
if torch.cuda.is_available():    

    # Tell PyTorch to use the GPU.    
    device = torch.device("cuda")

    print('There are %d GPU(s) available.' % torch.cuda.device_count())

    print('We will use the GPU:', torch.cuda.get_device_name(0))

# If not...
else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

Found GPU at: /device:GPU:0
There are 1 GPU(s) available.
We will use the GPU: Tesla P4


In [13]:
import json
with open("Data/data_sample_10.json") as infile:
  data = json.load(infile)
print(data['1022'])

{'prompt': 'We (family friends) planned a holiday trip to go for the last weekend. I was preparing for that like packing dress and getting ready to it. Unfortunately one of my cousin was unable to come due to his personal reasons. So we just cancelled the trip. I was feeling sad for this trip cancellation. ', 'emotion': 'sad', 'dialog': [{'speaker': 43, 'utterance': 'My holiday trip has cancelled. I was feeling low for it. '}, {'speaker': 167, 'utterance': 'That is bad news and hard to deal with.'}, {'speaker': 43, 'utterance': 'yeah. Waiting for the next holiday.'}, {'speaker': 167, 'utterance': 'I sure hope you have a wonderful next Holiday trip very soon.'}, {'speaker': 43, 'utterance': 'thank you! have a nice day'}]}


In [16]:
print(data.keys())
print(data['2103'])

dict_keys(['1022', '2103', '4072', '9710', '11290', '11572', '13042', '13485', '18577', '21184'])
{'prompt': 'My cousin bought a really pretty shirt that I had been wanting.', 'emotion': 'jealous', 'dialog': [{'speaker': 226, 'utterance': 'My cousin bought a really pretty shirt that I had been wanting'}, {'speaker': 10, 'utterance': 'My cousin bought a really pretty shirt that I had been wanting'}, {'speaker': 226, 'utterance': 'that is not fun_comma_ does that mean you cannot get it now?'}, {'speaker': 10, 'utterance': "No I can but I didn't want her to have it and not me."}, {'speaker': 226, 'utterance': 'that is no fun_comma_ does that mean you cannot buy it now?'}, {'speaker': 10, 'utterance': "No I can but I didn't want her to have it and not me."}]}


In [0]:
with open("Data/data_fixed_train.json","r") as infile:
  data = json.load(infile)
  with open("Data/data_complement_analysis.csv","w") as outfile:
    outfile.write("emotion|speaker_utterance|listener_utterance\n")
    for key in data.keys():
      dialog = data[key]['dialog']
      emotion = data[key]['emotion']
      i = 0
      while i < len(dialog):
        speaker_utterance = str(dialog[i]['utterance']).replace("|","").replace("_comma_",",")
        listener_utterance = str(dialog[i+1]['utterance']).replace("|","").replace("_comma_",",") if i + 1 < len(dialog) else ""
        line = "%s|%s|%s\n" % (emotion, speaker_utterance, listener_utterance)
        outfile.write(line)
        #print(line)
        i += 2